## Target Obs Stratgey



> First, we need to import some packages in Julia

In [1]:
using Pkg
Pkg.activate("../.."); # Actives the environment at the root of the project
# Using our Carbon-I and vSmartMOM packages (developed by our team)
using CarbonI
using Dates, DelimitedFiles, XLSX, DataFrames

  Activating project at `~/code/gitHub/CarbonI`
┌ Info: CUDA-enabled GPU(s) detected
└ @ vSmartMOM /home/cfranken/.julia/packages/vSmartMOM/a7WZC/src/vSmartMOM.jl:57
┌ Info: CUDA.CuDevice(0): NVIDIA A100-PCIE-40GB
└ @ vSmartMOM /home/cfranken/.julia/packages/vSmartMOM/a7WZC/src/vSmartMOM.jl:59
┌ Info: CUDA.CuDevice(1): NVIDIA A100-PCIE-40GB
└ @ vSmartMOM /home/cfranken/.julia/packages/vSmartMOM/a7WZC/src/vSmartMOM.jl:59


In [31]:
using  CairoMakie, Interpolations
include(joinpath("../../src/Plots", "CI_colorsNew.jl"))
# Read the Excel data
filename = "../../data/Plant.csv"

# The Load CARMA data
# The readtable function returns a tuple; we convert it to a DataFrame.
data = readdlm(filename,',', skipstart=1)
emi = data[:,4]
emi[emi.==""] .= NaN

emissions_annual = convert.(Float64,emi);

# Load CH4 from Airborne Campaigns (Cusworth PNAS paper)
filename = "../../data/carbonmapper_ch4_plumelist_2020_2021.xlsx"
sheetname = "carbonmapper_ch4_plumelist_2020"
# The readtable function returns a tuple; we convert it to a DataFrame.
data = DataFrame(XLSX.readtable(filename, sheetname))
plumes_ch4 = convert.(Float64,data[!,"qplume"]);

In [32]:
# Process CARMA data
emissions_hourly_co2 = emissions_annual ./ 8760.0
# Clean NaNs
emissions_hourly_co2_clean = filter(!isnan, emissions_hourly_co2)

# 3. Sort the hourly emissions array in ascending order (for a CDF).
sorted_emissions = sort(emissions_hourly_co2_clean, rev=true)
sorted_emissions_rev = sort(emissions_hourly_co2_clean, rev=false)

# 4. Compute the cumulative sum and normalize by the total. This gives an
#    empirical cumulative distribution function (CDF) of total emissions.
cdf_vals = cumsum(sorted_emissions) ./ sum(sorted_emissions)
cdf_vals_rev = cumsum(sorted_emissions_rev) ./ sum(sorted_emissions_rev)

inter_cdf = LinearInterpolation( 100cdf_vals, sorted_emissions)
inter_cdf_rev = LinearInterpolation(  sorted_emissions_rev[sorted_emissions_rev.>0],100cdf_vals_rev[sorted_emissions_rev.>0]);

In [33]:
# Process CH4:

# 3. Sort the hourly emissions array in ascending order (for a CDF).
sorted_emissions_ch4 = sort(plumes_ch4, rev=true)
sorted_emissions_rev_ch4 = sort(plumes_ch4, rev=false)

# 4. Compute the cumulative sum and normalize by the total. This gives an
#    empirical cumulative distribution function (CDF) of total emissions.
cdf_vals_ch4 = cumsum(sorted_emissions_ch4) ./ sum(sorted_emissions_ch4)
cdf_vals_rev_ch4 = cumsum(sorted_emissions_rev_ch4) ./ sum(sorted_emissions_rev_ch4)

inter_cdf_ch4 = LinearInterpolation( 100cdf_vals_ch4, sorted_emissions_ch4)
inter_cdf_rev_ch4 = LinearInterpolation(  sorted_emissions_rev_ch4[sorted_emissions_rev_ch4.>0],100cdf_vals_rev_ch4[sorted_emissions_rev_ch4.>0]);


In [34]:
sorted_emissions_ch4

2526-element Vector{Float64}:
 9697.235917
 6522.335382
 5774.344132
 5506.323639
 5485.087488
 5366.898563
 4978.777499
 4184.124144
 4180.863976
 4114.793208
    ⋮
   19.55850618
   18.85841358
   18.80043312
   15.38239871
   12.38318077
   11.81340847
   11.10234465
    8.354539236
    8.091911213

In [35]:
# Specify requirements for CO2
req_global = 100
req_target = 35

cbe_global = 57
cbe_target = 18

# What fraction of plumes we want to detect:
req_percent = 85.0

85.0

In [42]:
# Specify requirements for CH4
req_global_ch4 = 175
req_target_ch4 = 62

cbe_global_ch4 = 99
cbe_target_ch4 = 31

# What fraction of plumes we want to detect:
req_percent_ch4 = 90.0

90.0

In [ ]:

set_theme!(theme_ggplot2())
# 5. Create a Figure and an Axis for the plot using CairoMakie.
fig = Figure(resolution=(700,800))
ax = Axis(fig[1,1];
    ylabel = "Detection Limit of CO₂ Emissions (tons/hour)",
    
    title = "From Coverage Requirements to Detection Limits",
    yscale = log10,
    xticks = 0:10:100,
    yminorgridvisible=true,
    #xminorgridvisible=true,
    #xminorticks = IntervalsBetween(5),
    yminorticks = IntervalsBetween(5)
    )

req_flux = inter_cdf(req_percent)
cbe_percent = 100 .- inter_cdf_rev(cbe_global)
@show cbe_percent
# 6. Plot the data as a line. Use `lines!` or `steppre!`/`steppost!` for a classic stepwise ECDF.
lines!(ax,  100cdf_vals,sorted_emissions,color = CarbonI_colors[4], linewidth=3)
#lines!(ax,  [req_percent,req_percent],[10,inter_cdf(req_percent)], color = :red)
arrows!(ax, [req_percent,],[10,],[0,],[req_flux-20,],arrowsize = 10, linewidth=2,color = CarbonI_colors[10])
arrows!(ax, [req_percent,],[req_flux,],[-req_percent+1,],[0,],arrowsize = 10, linewidth=2,color = CarbonI_colors[10])
text!(ax, " Req. >85%", position = (req_percent, 25),
      align = (:center, :bottom), color = CarbonI_colors[10], rotation=π/2)

#@show inter_cdf(req_percent)
text!(ax, "Req. Detection limit <  $(round(Int, req_flux)) tons/hr ", position = (req_percent/2,req_flux),
      align = (:right, :bottom), color = CarbonI_colors[10])

arrows!(ax, [0,],[cbe_global,],[cbe_percent-1,],[0,],arrowsize = 10, linewidth=2,color = CarbonI_colors[10], linestyle = :dash)
arrows!(ax, [cbe_percent,],[cbe_global,],[0,],[-cbe_global+11,],arrowsize = 10, linewidth=2,color = CarbonI_colors[10],linestyle = :dash)
text!(ax, "CBE >$(round(Int, cbe_percent))%", position = (cbe_percent, 25),
    align = (:center, :bottom), color = CarbonI_colors[10], rotation=π/2)
      
#@show inter_cdf(req_percent)
text!(ax, "CBE Detection limit <  $(round(Int, cbe_global)) tons/hr ", position = (cbe_percent/2,cbe_global),
            align = (:right, :bottom), color = CarbonI_colors[10])

#text!(ax, "50th Percentile", position = (arrow_x, arrow_tip_y + 10),
   #   align = (:center, :bottom), color = :red)
ylims!(ax, 10, 10000)
xlims!(ax, 0, 100)
hidexdecorations!(ax, grid=false)

ax2 = Axis(fig[2,1];
    ylabel = "Detection Limit of CH₄ Emissions (kg/hour)",
    xlabel = "Required Fraction of Total Emissions (%)",
    #title = "Empirical CDF of Hourly CO₂ Emissions",
    yscale = log10,
    xticks = 0:10:100,
    yminorgridvisible=true,
    #xminorgridvisible=true,
    #xminorticks = IntervalsBetween(5),
    yminorticks = IntervalsBetween(5)
    )

req_flux_ch4 = inter_cdf_ch4(req_percent_ch4)
cbe_percent_ch4 = 100 .- inter_cdf_rev_ch4(cbe_global_ch4)
@show cbe_percent
# 6. Plot the data as a line. Use `lines!` or `steppre!`/`steppost!` for a classic stepwise ECDF.
lines!(ax2,  100cdf_vals_ch4,sorted_emissions_ch4,color = CarbonI_colors[4], linewidth=3)
#lines!(ax,  [req_percent,req_percent],[10,inter_cdf(req_percent)], color = :red)
arrows!(ax2, [req_percent_ch4,],[10,],[0,],[req_flux_ch4-20,],arrowsize = 10, linewidth=2,color = CarbonI_colors[10])
arrows!(ax2, [req_percent_ch4,],[req_flux_ch4,],[-req_percent_ch4+1,],[0,],arrowsize = 10, linewidth=2,color = CarbonI_colors[10])
text!(ax2, "Req. >90%", position = (req_percent_ch4, 25),
      align = (:center, :bottom), color = CarbonI_colors[10], rotation=π/2)

#@show inter_cdf(req_percent)
text!(ax2, "Req. Detection limit <  $(round(Int, req_flux_ch4)) kg/hr ", position = (req_percent_ch4/2,req_flux_ch4),
      align = (:right, :bottom), color = CarbonI_colors[10])

arrows!(ax2, [0,],[cbe_global_ch4,],[cbe_percent_ch4-1,],[0,],arrowsize = 10, linewidth=2,color = CarbonI_colors[10], linestyle = :dash)
arrows!(ax2, [cbe_percent_ch4,],[cbe_global_ch4,],[0,],[-cbe_global_ch4+11,],arrowsize = 10, linewidth=2,color = CarbonI_colors[10],linestyle = :dash)
text!(ax2, "CBE  >$(round(Int, cbe_percent_ch4))%", position = (cbe_percent_ch4, 25),
    align = (:center, :bottom), color = CarbonI_colors[10], rotation=π/2)
      
#@show inter_cdf(req_percent)
text!(ax2, "CBE Detection limit <  $(round(Int, cbe_global_ch4)) kg/hr ", position = (cbe_percent_ch4/2,cbe_global_ch4),
            align = (:right, :bottom), color = CarbonI_colors[10])
text!(ax2, "CH₄", position = (90, 4e3),
            align = (:left, :bottom), color = :black, fontsize=25)
text!(ax, "CO₂", position = (90, 4e3),
            align = (:left, :bottom), color = :black, fontsize=25)
#text!(ax, "50th Percentile", position = (arrow_x, arrow_tip_y + 10),
   #   align = (:center, :bottom), color = :red)
ylims!(ax2, 10, 10000)
xlims!(ax2, 0, 100)

fig     

MethodError: MethodError: no method matching initialize_block!(::Axis; minorticks::IntervalsBetween)
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

Closest candidates are:
  initialize_block!(::Axis; palette) got unsupported keyword argument "minorticks"
   @ Makie ~/.julia/packages/Makie/RgxaV/src/makielayout/blocks/axis.jl:160
  initialize_block!(!Matched::SliderGrid, NamedTuple...) got unsupported keyword argument "minorticks"
   @ Makie ~/.julia/packages/Makie/RgxaV/src/makielayout/blocks/slidergrid.jl:47
  initialize_block!(!Matched::IntervalSlider) got unsupported keyword argument "minorticks"
   @ Makie ~/.julia/packages/Makie/RgxaV/src/makielayout/blocks/intervalslider.jl:1
  ...


In [58]:
save("../../plots/CO2_CH4_Flux_Detection.pdf", fig)

CairoMakie.Screen{PDF}


In [41]:
@show req_flux_ch4

242.14253199255876